In [1]:
rpath = '../data/raw/'
wpath = '../data/tyc/'

抽取公司字典（url->名称）、每个公司的投资机构：

In [2]:
comp_dict = {}
org_comp = {}

for line in open(rpath + 'company_total.json', encoding='utf-8'):
    line = eval(line)
    comp_id, comp_name = line['id'], line['名称']
    comp_dict[comp_id] = comp_name
    
    org = line.get('投资机构')
    # 投资机构不存在或者为空列表
    if not org:
        continue
    org_comp[org[0]] = comp_id

In [3]:
len(comp_dict), len(org_comp)

(92524, 2441)

这样抽取只有2k多个公司有投资机构，需要通过投资机构的基金管理人扩充公司和机构的对应关系。

从机构信息中抽取基金管理人、投资事件（公开和未公开）：

In [4]:
org_dict = {}
org_manager = {}
org_gk = {}
org_wgk = {}

for line in open(rpath + 'organize_all.json', encoding='utf-8'):
    line = eval(line)
    org_id, man, gk, wgk = line['id'], line.get('基金管理人'), line.get('公开投资事件'), line.get('未公开投资')
    org_dict[org_id] = line.get('名称')
    if man:
        org_manager[org_id] = [c['企业名称ID'] for c in man]
    if gk:
        org_gk[org_id] = [c['CompanyID'] for c in gk if 'CompanyID' in c]
    if wgk:
        org_wgk[org_id] = [c['企业名称ID'] for c in wgk]

In [5]:
len(org_dict), len(org_manager), len(org_gk), len(org_wgk)

(14290, 5691, 13336, 10127)

org_id 与 comp_id 一一对应，将company和organize数据的comp-org对应关系合并

In [6]:
org_comp_total = org_comp
for k, v in org_manager.items():
    org_comp_total[k] = v[0]

In [7]:
len(org_comp_total)

7345

用工商追踪数据补充投资事件：

In [8]:
org_gszz = {}

for line in open(rpath + 'organize_all.json', encoding='utf-8'):
    line = eval(line)
    org_id, gszz = line['id'], line.get('工商追踪')
    if gszz:
        org_gszz[org_id] = []
        for rec in gszz:
            content = rec['内容']
            if '对外投资' in content['文本']:
                org_gszz[org_id].append((content['CompanyID'], content['公司名称'], rec['投资日期'], content.get('投资比例')))

In [9]:
sum(len(v) for v in org_gszz.values())

96205

为公司编号：

In [10]:
comp_list = list(org_comp_total.values())
for c in org_gk.values():
    comp_list.extend(c)
for c in org_wgk.values():
    comp_list.extend(c)
    
for rec in org_gszz.values():
    comp_list.extend(list(list(zip(*rec))[0]))

comp_list = list(set(c for c in comp_list if 'company' in c) & comp_dict.keys())
comp_list.sort()
comp_ind = {v: k for k, v in enumerate(comp_list)}

保存公司标号、url和名称的对应：

In [11]:
import pandas as pd

df = pd.DataFrame({ 'cid': comp_list, 'cname': [comp_dict[c] for c in comp_list] })
df.to_csv(wpath + 'comps.csv', index=False)

In [12]:
pd.read_csv(wpath + 'comps.csv')

,cid,cname
0,company/1000025524,安徽长丰农村商业银行股份有限公司
1,company/1000031266,成都全搜索科技有限责任公司
2,company/1000368393,安徽三联机器人科技有限公司
3,company/1000406554,广州中包联外经实业有限公司
4,company/1000516298,成都吉胜科技有限责任公司
...,...,...
64421,company/99780228,北京北比信息技术有限公司
64422,company/998424134,约印大通（天津）资产管理有限公司
64423,company/99871402,北京红龙文化传媒有限公司
64424,company/998961890,福建省蓝深环保技术股份有限公司


所有投资事件，包括工商追踪和公开投资（未公开投资没有时间）：

In [13]:
touzi_all = {}        

for line in open(rpath + 'organize_all.json', encoding='utf-8'):
    line = eval(line)
    org_id, gszz, gktz = line['id'], line.get('工商追踪'), line.get('公开投资事件')
    if org_id not in org_comp_total:
        # print(org_id)
        continue

    if gszz:
        for rec in gszz:
            content = rec['内容']
            if '对外投资' in content['文本']:
                touzi_all[(org_comp_total[org_id], content['CompanyID'])] = rec['投资日期']

    if gktz:
        for rec in gktz:
            if 'CompanyID' not in rec:
                continue
            touzi_all[(org_comp_total[org_id], rec['CompanyID'])] = rec['投资时间']

In [14]:
import csv

with open(wpath + 'comp_touzi_comp.csv', 'w', encoding='utf-8') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['src_cid', 'src_cname', 'src_ind', 'dst_cid', 'dst_cname', 'dst_ind', 'date'])
    for (src_cid, dst_cid), date in touzi_all.items():
        if src_cid not in comp_ind or dst_cid not in comp_ind:
            continue
        if date < '1950-01-01':
            continue
        writer.writerow([src_cid, comp_dict[src_cid], comp_ind[src_cid], dst_cid, comp_dict[dst_cid], comp_ind[dst_cid], date])